In [ ]:
# VERSIUNEA FINALA A PROIECTULUI

In [ ]:
# !! Celulele cu cod se reuleaza in ordine 

In [35]:
import cv2 as cv
import numpy as np
import os

In [36]:
bottom_left = [ 786, 2877]
bottom_right = [2424, 2862]
top_left = [ 777, 1211]
top_right = [2413, 1212]

In [37]:
def get_careu(image):
   
    image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)

    width = 2000
    height = 2000

    image_copy = cv.cvtColor(image.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)

    puzzle = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32')
    destination = np.array([[0,0], [width, 0], [width, height], [0, height]], dtype='float32')

    M = cv.getPerspectiveTransform(puzzle, destination)
    result = cv.warpPerspective(image, M, (width, height))
    # result = cv.cvtColor(result,cv.COLOR_BGR2GRAY)

    return result

In [38]:
lines_horizontal=[]
for i in range(0,2000,133):
    l=[]
    l.append((0,i))
    l.append((1998,i))
    lines_horizontal.append(l)

In [39]:
lines_vertical=[]
for i in range(0,2000,133):
    l=[]
    l.append((i,0))
    l.append((i,1998))
    lines_vertical.append(l)

In [40]:
def clasifica_cifra(patch, path):
        maxi=-np.inf
        poz=-1
        file_poz = ''
        files=os.listdir(path)
        for file in files:
            img_template=cv.imread(path+file)
            img_template= cv.cvtColor(img_template,cv.COLOR_BGR2GRAY)
            thresh, binary = cv.threshold(img_template, 150, 255, cv.THRESH_BINARY)
            binary_resize = cv.resize(binary, (134,134))
            corr = cv.matchTemplate(patch,binary_resize,cv.TM_CCOEFF_NORMED)
            corr=np.max(corr)
            if corr>maxi:
                maxi=corr
                file_poz = file
                poz=int(file[0])
        return poz, file_poz

In [41]:
def determina_configuratie_careu(thresh,lines_horizontal,lines_vertical):
    matrix = np.empty((15,15), dtype='str')
    for i in range(len(lines_horizontal)-1):
        for j in range(len(lines_vertical)-1):
            y_min = lines_vertical[j][0][0] + 10
            y_max = lines_vertical[j + 1][1][0] - 10
            x_min = lines_horizontal[i][0][1] + 10
            x_max = lines_horizontal[i + 1][1][1] - 10
            patch = thresh[x_min:x_max, y_min:y_max].copy()
            
            medie = np.mean(patch)
            
            if medie > 140:
                path = 'templates/all/'
                poz, file = clasifica_cifra(patch, path)
                if (poz == 7 or poz == 8) and (i == 7 or i == 6) and (j == 7 or j == 8):
                    matrix[i][j] = '-'
                elif poz == 7 or poz == 8:
                    path2 = 'templates/dice faces/'
                    matrix[i][j],_ = clasifica_cifra(patch, path2)
                else:
                    matrix[i][j] = poz
            else:
                matrix[i][j] = '-'
            
    return matrix

In [42]:
# AFISARE IN FISIER
def task1_and_task2(img_path, sol_path):
    pozitii_piese = []
    dict_poz = {1:'A', 2:'B', 3:'C', 4:'D', 5:'E', 6:'F', 7:'G', 8:'H', 9:'I', 10:'J', 11:'K', 12:'L', 13:'M', 14:'N', 15:'O'}
    game_number = 1
    
    files=os.listdir(img_path)
    for file in files:
        
        if game_number != int(file[0]):
            game_number = int(file[0])
            pozitii_piese = []
            
        if file[-3:]=='jpg':
            img = cv.imread(img_path +'/'+file)
            careu = get_careu(img)
            
            thresh, binary = cv.threshold(careu, 150, 255, cv.THRESH_BINARY)
            matrice=determina_configuratie_careu(binary,lines_horizontal,lines_vertical)
    
            with open(sol_path + '/{f}.txt'.format(f=file[:-4]), 'w') as f:
                for i in range(15):
                    for j in range(15):
                        poz = [(i,j)]
                        if matrice[i][j] != '-' and poz not in pozitii_piese:
                            f.write("{ii}{jj} ".format(ii=i+1, jj=dict_poz[j+1]))
                            f.write("{nr}".format(nr=matrice[i][j]))
                            f.write("\n")
                            pozitii_piese.append(poz)

In [43]:
task1_and_task2('antrenare', 'verificare/antrenare')

In [44]:
# TASK 3

In [45]:
score_route = [-1, 1, 2, 3, 4, 5, 6, 0, 2, 5, 3, 4, 6, 2, 2, 0, 3, 5, 4, 1, 6, 2, 4, 5, 5, 0, 6, 3, 4, 2, 0, 1, 5, 1, 3, 4, 4, 4, 5, 0, 6, 3, 5, 4, 1, 3, 2, 0, 0, 1, 1, 2, 3, 6, 3, 5, 2, 1, 0, 6, 6, 5, 2, 1, 2, 5, 0, 3, 3, 5, 0, 6, 1, 4, 0, 6, 3, 5, 1, 4, 2, 6, 2, 3, 1, 6, 5, 6, 2, 0, 4, 0, 1, 6, 4, 4, 1, 6, 6, 3]

special_patches = [('1A', 5), ('15A', 5), ('1O', 5), ('15O', 5), ('4A', 4), ('12A', 4), ('6B', 4), ('12B', 4), ('1D', 4), ('15D', 4), ('2F', 4), ('14F', 4), ('2J', 4), ('14J', 4), ('1L', 4), ('15L', 4), ('6N', 4), ('10N', 4), ('4O', 4), ('12O', 4), ('8A', 3), ('3B', 3), ('13B', 3), ('2C', 3), ('14C', 3), ('4D', 3), ('12D', 3), ('1H', 3), ('15H', 3), ('4L', 3), ('12L', 3), ('2M', 3), ('14M', 3), ('3N', 3), ('13N', 3), ('8O', 3), ('5C', 2), ('11C', 2), ('6D', 2), ('10D', 2), ('3E', 2), ('13E', 2), ('4F', 2), ('12F', 2), ('4J', 2), ('12J', 2), ('3K', 2), ('13K', 2), ('6L', 2), ('10L', 2), ('5M', 2), ('11M', 2), ('5E', 1), ('7E', 1), ('9E', 1), ('11E', 1), ('6F', 1), ('10F', 1), ('5G', 1), ('11G', 1), ('5I', 1), ('11I', 1), ('6J', 1), ('10J', 1), ('5K', 1), ('7K', 1), ('9K', 1), ('11K', 1)]

In [46]:
# extrag pozitia si numarul celor 2 patch-uri ale piesei de domino
def get_pieces(image, sol_path):
    path = sol_path + '/' + image[:-3] +'txt'
    with open (path, 'r') as g:
        line1 = g.readline().split(" ")
        line2 = g.readline().split(" ")
        g.close()
    return  line1[0], line1[1], line2[0], line2[1]

In [47]:
# extrag punctele aferente unei pozitii
def get_points_piece_position(piece_poz):
    piece_point = 0
    for special_patch in special_patches:
        if piece_poz == special_patch[0]:
            piece_point = special_patch[1]
    return piece_point

In [48]:
# adaug scorul in fisier
def write_score(image, scor, sol_path):
    path = sol_path + '/' + image[:-3] +'txt'
    with open (path, 'a') as g:
        scor = str(scor)
        g.write(scor)
        g.close()

In [49]:
# task 3 - functie ce determina punctajul unui joc
def get_game_score(game_number, img_path, sol_path):
    global score_route, special_patches
    
    player_1_route_poz = 0
    player_2_route_poz = 0
    
    file = '{gn}_mutari.txt'.format(gn=game_number)
    path = img_path + '/' + file
    with open (path, 'r') as f:
        for i in range(20):
            line = f.readline().split(" ")
            image = line[0]
            player = line[1]
    
            current_player_score = 0
            
            poz1, nr1, poz2, nr2 = get_pieces(image, sol_path)
            
            point_piece1 = get_points_piece_position(poz1)
            point_piece2 = get_points_piece_position(poz2)
            
            # daca punem un domino dublu pe un patch special primim dublu puncte
            if nr1 == nr2:
                if point_piece1 != 0:
                    current_player_score = 2*point_piece1
                elif point_piece2 != 0:
                    current_player_score = 2*point_piece2
            else:
                if point_piece1 != 0:
                    current_player_score += point_piece1
                elif point_piece2 != 0:
                    current_player_score += point_piece2
                    
            # vedem ce player joaca 
            if player[-2] == "1":
                if str(score_route[player_1_route_poz]) == nr1[0] or str(score_route[player_1_route_poz]) == nr2[0]:
                    current_player_score += 3

                if str(score_route[player_2_route_poz]) == nr1[0] or str(score_route[player_2_route_poz]) == nr2[0]:
                   player_2_route_poz += 3
            else:
                if str(score_route[player_2_route_poz]) == nr1[0] or str(score_route[player_2_route_poz]) == nr2[0]:
                    current_player_score += 3

                if str(score_route[player_1_route_poz]) == nr1[0] or str(score_route[player_1_route_poz]) == nr2[0]:
                   player_1_route_poz += 3
            
            # adaug punctele pozitiei player-ului de pe traseul de joc
            if player[-2] == "1":
                player_1_route_poz += current_player_score
            else:
                player_2_route_poz += current_player_score

            # scriem scorul in fisier
            write_score(image, current_player_score, sol_path)

In [50]:
# apelez functia pentru toate cele 5 jocuri
for i in range(5):
    get_game_score(i+1, 'antrenare', 'verificare/antrenare')

In [ ]:
# if necessary
# how i identified the coordinates of the game board 
# Corners: 
# Bottom left: [ 786 2877]
# Bottom right: [2424 2862]
# Top left: [ 777 1211]
# Top right: [2413 1212]

# image = cv.imread('antrenare/1_01.jpg')
# 
# image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
# 
# image_m_blur = cv.medianBlur(image,3)
# show_image("Median Blur",image_m_blur)
# 
# image_g_blur = cv.GaussianBlur(image, (0,0), 7) 
# show_image("Gaussian Blur", image_g_blur)
# 
# image_sharpened = cv.addWeighted(image_m_blur, 1.5, image_g_blur, -0.5, 0)
# show_image('image_sharpened',image_sharpened)
# _, thresh = cv.threshold(image_sharpened, 30, 200, cv.THRESH_BINARY)
# 
# kernel = np.ones((5,5), np.uint8)
# thresh = cv.erode(thresh, kernel)
# show_image('image_thresholded',thresh)
# 
# edges =  cv.Canny(thresh ,300,400)
# show_image('edges',edges)
# 
# contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
# max_area = 0
#     
# for i in range(len(contours)):
#     if(len(contours[i]) >3):
#         possible_top_left = None
#         possible_bottom_right = None
#         for point in contours[i].squeeze():
#             if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
#                 possible_top_left = point
# 
#             if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
#                 possible_bottom_right = point
# 
#         diff = np.diff(contours[i].squeeze(), axis = 1)
#         possible_top_right = contours[i].squeeze()[np.argmin(diff)]
#         possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
#         if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
#             max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
#             top_left = possible_top_left
#             bottom_right = possible_bottom_right
#             top_right = possible_top_right
#             bottom_left = possible_bottom_left
#             
# 
# width = 2000
# height = 2000
# # bottom_right[0] = top_right[0]
# # bottom_right[1] = bottom_left[1]
# 
# image_copy = cv.cvtColor(image.copy(),cv.COLOR_GRAY2BGR)
# cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
# cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
# cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
# cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
# show_image("detected corners",image_copy)
# 
# print("Corners: ")
# print("Bottom left: " + str(bottom_left))
# print("Bottom right: " + str(bottom_right))
# print("Top left: " + str(top_left))
# print("Top right: " + str(top_right))
# 
# puzzle = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32')
# destination = np.array([[0,0], [width, 0], [width, height], [0, height]], dtype='float32')
# 
# M = cv.getPerspectiveTransform(puzzle, destination)
# result = cv.warpPerspective(image, M, (width, height))
# 
# show_image("rezultat", result)